# Phân loại Titanic bằng Gaussian Naive Bayes
Dataset: Titanic_train.csv, Titanic_test.csv

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

## Đọc dữ liệu

In [2]:
train_df = pd.read_csv("Titanic_train.csv")
test_df = pd.read_csv("Titanic_test.csv")

print("Kích thước train:", train_df.shape)
print("Kích thước test:", test_df.shape)
train_df.head()

Kích thước train: (891, 12)
Kích thước test: (418, 11)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## EDA cơ bản

In [3]:
print("Thông tin dữ liệu:")
print(train_df.info())

print("\nGiá trị thiếu:")
print(train_df.isnull().sum())

print("\nThống kê mô tả Age & Fare:")
print(train_df[['Age', 'Fare']].describe())

print("\nTỉ lệ sống sót:")
print(train_df['Survived'].value_counts(normalize=True))

Thông tin dữ liệu:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None

Giá trị thiếu:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin         

## Tiền xử lý dữ liệu
- Điền giá trị thiếu cho Age bằng trung bình
- Điền giá trị thiếu cho Embarked bằng mode
- Chuyển Sex (male=0, female=1)
- One-hot cho Embarked
- Giữ các đặc trưng quan trọng: Pclass, Sex, Age, SibSp, Parch, Fare, Embarked

In [8]:
def preprocess(df):
    df = df.copy()
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    df['Fare'] = df['Fare'].fillna(df['Fare'].mean())
    df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])

    df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

    df = pd.get_dummies(df, columns=['Embarked'], drop_first=True)

    features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 
                'Embarked_Q', 'Embarked_S']
    return df[features]


## Chia dữ liệu train/test và huấn luyện Gaussian Naive Bayes

In [9]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

nb_clf = GaussianNB()
nb_clf.fit(X_train, y_train)

GaussianNB()

## Đánh giá mô hình

In [10]:
y_pred = nb_clf.predict(X_val)

print("Accuracy:", accuracy_score(y_val, y_pred))
print("\nClassification Report:\n", classification_report(y_val, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_val, y_pred))

Accuracy: 0.770949720670391

Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.80      0.80       105
           1       0.72      0.73      0.72        74

    accuracy                           0.77       179
   macro avg       0.76      0.76      0.76       179
weighted avg       0.77      0.77      0.77       179


Confusion Matrix:
 [[84 21]
 [20 54]]


## Dự đoán trên Titanic_test.csv

In [11]:
X_test = preprocess(test_df)
test_preds = nb_clf.predict(X_test)

print("Dự đoán trên tập test (5 mẫu đầu):", test_preds[:5])

Dự đoán trên tập test (5 mẫu đầu): [0 1 0 0 1]


## Nhận xét
- Naive Bayes khá đơn giản, kết quả thường ~75–80% trên Titanic dataset.
- Các đặc trưng quan trọng nhất: Sex, Pclass, Age.
- Hạn chế: giả định độc lập giữa các biến có thể làm giảm độ chính xác.